In [1]:
import jsonlines
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
import torch.nn.functional as F
import os
import copy
import json
import pickle


os.environ['CUDA_VISIBLE_DEVICES'] = '0'
def file_reader(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        src = []
        tgt = []
        for obj in reader:
            src.append(obj['src'])
            tgt.append(obj['tgt'])
    return src, tgt

def get_query(input):
    query = []
    src = []
    for item in input:
        temp = item.split('</s>')
        query.append(temp[0].replace('<s>','').strip())
        src.append(temp[1].replace('<s>','').strip())
    return query, src

# Segment 文章到 Paragraph 中

In [3]:
src, tgt = file_reader('processed_data_turn_split/train.jsonl')
querys, src = get_query(src)
max_input_len = 512
number_of_segment = 16
src = [i.split('<turn_seperator>') for i in src]

segmented_input = []
non_segmented_input = []

In [4]:
def extract_names(s):
    name = ''
    sentence = ''
    flag = 0
    for w in s:
        if w != ':' and flag == 0:
            name += w
        if w == ':':
            flag = 1
        elif flag == 1:
            sentence += w
    return name, sentence

In [5]:
def read_conversation(data):
    conversations = []

    for i in range(0, len(data)):
        if len(data[i]['dialogue'].split('\r\n')) > 1:
            sentences = data[i]['dialogue'].replace("|", " ").split('\r\n')
            
        else:
            sentences = data[i]['dialogue'].replace("|", " ").split('\n')
            
        if len(sentences) == 1:
            continue
            
        conversations.append(sentences)
        print(sentences)
        break
    return conversations

In [22]:
import re
from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
import allennlp_models.structured_prediction
import os
import spacy
nlp = spacy.blank("en")
nlp.add_pipe(nlp.create_pipe('sentencizer'))
# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)

# You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.
def coreference(s):
    doc = nlp(s)
    return doc._.coref_clusters

In [7]:
def find_nearest_coref(full_conv, index, word):
    ref_list = ['he', 'she', 'him', 'her', 'it', 'his']
    prev_sentence = ''
    for i in range(index, -1, -1):
        #print(i)
        cur_sentence =  full_conv[i][1]
        prev_sentence = cur_sentence + ' ' + prev_sentence
        coref = coreference(prev_sentence)
        #print('----')
        #print(prev_sentence)
        #print(coref)
        #print('----')
        if len(coref) > 0:
            for clusters in coref:
                if word in [str(w) for w in clusters.mentions] and str(clusters.main).lower() not in ref_list:
                    return str(clusters.main)
    
    return word

In [8]:
from nltk.tokenize import word_tokenize
import string
punc = string.punctuation

In [9]:
first = ['i', 'me', 'my']
second = ['you', 'u', 'your']
third = ['he', 'she', 'him', 'her', 'it', 'his']

def transform_perspective(conv):
    transformed_conv = ''
    temp = []
    flag = 0
    
    
    for i in range(0, len(conv)):
        #print(i)
        cur_name = conv[i][0]
        cur_uttr = conv[i][1]
        s = ''
        

        for word in word_tokenize(cur_uttr):
            if word.lower() in first:
                temp_word = cur_name
            elif word.lower() in second:
                try:
                    temp_word = conv[i+1][0]
                except:
                    temp_word = conv[i-1][0]
            elif word.lower() in third:
                temp_word = find_nearest_coref(conv, i, word)
            else:
                temp_word = word

            s = s + temp_word + ' '

        #j = len(s)-1
        #while s[j] == ' ':
        #    j = j - 1

        #if s[j] not in punc:
        s += '.'


        temp.append(s)
                
                
        
    transformed_conv = ' '.join(temp)

    return transformed_conv, temp

In [10]:
import re

alphabets = "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|me|edu)"

def sentence_tokenize(text):
    text = " " + text + "  "
    text = text.replace("\n", " ")
    text = re.sub(prefixes, "\\1<prd>", text)
    text = re.sub(websites, "<prd>\\1", text)
    if "Ph.D" in text:
        text = text.replace("Ph.D.", "Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] ", " \\1<prd> ", text)
    text = re.sub(acronyms+" "+starters, "\\1<stop> \\2", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" +
                  alphabets + "[.]", "\\1<prd>\\2<prd>\\3<prd>", text)
    text = re.sub(alphabets + "[.]" + alphabets +
                  "[.]", "\\1<prd>\\2<prd>", text)
    text = re.sub(" "+suffixes+"[.] "+starters, " \\1<stop> \\2", text)
    text = re.sub(" "+suffixes+"[.]", " \\1<prd>", text)
    text = re.sub(" " + alphabets + "[.]", " \\1<prd>", text)

    text = re.sub("([0-9])" + "[.]" + "([0-9])", "\\1<prd>\\2", text)

    if "..." in text:
        text = text.replace("...", "<prd><prd><prd>")
    if "”" in text:
        text = text.replace(".”", "”.")
    if "\"" in text:
        text = text.replace(".\"", "\".")
    if "!" in text:
        text = text.replace("!\"", "\"!")
    if "?" in text:
        text = text.replace("?\"", "\"?")

    text = text.replace(".", ".<stop>")
    text = text.replace("?", "?<stop>")
    text = text.replace("!", "!<stop>")
    text = text.replace("<prd>", ".")

    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [25]:
from tqdm import tqdm
def transformat(data):
    convs = data
    processed_convs = []
    for i in range(0, len(convs)):
        temp = []
        conv = convs[i]
        name_prev = None
        same_sentence = ''
        for j in range(0, len(conv)):
            name, sentence = extract_names(conv[j])
            if name != name_prev:
                if name_prev is not None:
                    temp.append([name_prev, same_sentence])
                name_prev = name
                same_sentence = sentence
            elif name == name_prev:
                same_sentence = same_sentence + ' . ' + sentence

        temp.append([name_prev, same_sentence])

        processed_convs.append(temp)

    transformed_conv = []
    for j in tqdm(range(0, len(processed_convs))):
        # transformed_conv.append(transform_perspective(processed_convs[j])[1])
        print(transform_perspective(processed_convs[j])[1])
    return transformed_conv
    

In [26]:
transformed_conv = transformat(src)

  0%|          | 1/1257 [01:06<23:04:20, 66.13s/it]

['.', "yep . soon as project manager get this . okay . this is our last meeting . um project manager 'll go ahead and go through the minutes from the previous meeting . uh and then we 'll have a , the prototype presentation . um then we will um do an evaluation . uh or we 'll see what , what we need to have under the criteria for the evaluation . then we 'll go through the finance and see if we fall within the budget . um then we 'll do the evaluation , and then we can finish up after that with um any changes that we 'll need to make , or hopefully everything will fall right in line . um let 's see , minutes from the last meeting . um we looked at uh the the trends . we had uh the fashion trends that people want a fancy look-and-feel . it was twice as important as anything else . um they liked fruit and vegetables in the new styles . um and a spongy feel . so we were talking about trying to incorporate those into our prototype . um they wanted limited buttons and simplicity . um then w

  0%|          | 1/1257 [01:40<34:59:28, 100.29s/it]


KeyboardInterrupt: 

In [9]:
print(src[110])

['', 'user interface: here we go again . ', 'industrial designer: my mouse is not working anymore . ', 'user interface: oh . ', "industrial designer: he 's uh ", 'marketing: oh . ', 'user interface: okay . ', 'industrial designer: when i put it in , is is going to beep beep beep . ', 'marketing: oh , i got a nice little screen here over here . ', 'user interface: ', 'marketing: i got like this big black border uh on every side . ', 'industrial designer: ', 'project manager: okay . ', 'user interface: mm , okay . ', 'project manager: everybody ready ? ', "marketing: i 'll i 'll fix it . ", "user interface: yeah , it 's okay . ", 'project manager: welcome at the functional design meeting , again presented by maarten . ', 'marketing: yeah , whatever . ', 'user interface: ', 'marketing: ', "project manager: uh this is the agenda , the opening . uh , we 've got three presentations . and i 'm gon na show you some of the new projects requirements that were sent to me . and we 're gon na make 